In [1]:
import pandas as pd
import os
from pathlib import Path
import random
import re
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch

DSDIR = Path(os.environ["DSDIR"])

In [2]:
df = pd.read_csv(DSDIR / "meld_fidle/dev_sent_emo.csv")
df["question"] = ""
df["answer"] = ""
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,question,answer
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049",,
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261",,
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915",,
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960",,
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,1174,No.,Monica,sadness,negative,113,9,6,2,"00:19:28,792","00:19:29,876",,
1105,1175,What? Oh my God! Im gonna miss you so much!,Rachel,sadness,negative,113,10,6,2,"00:19:33,213","00:19:35,965",,
1106,1176,Im gonna miss you!,Monica,sadness,negative,113,11,6,2,"00:19:36,175","00:19:37,967",,
1107,1177,I mean its the end of an era!,Rachel,sadness,negative,113,12,6,2,"00:19:39,094","00:19:40,928",,


In [3]:
# Initialize the model and its tokenizer
model = AutoModelForCausalLM.from_pretrained(
    DSDIR / "HuggingFace_Models/meta-llama/Meta-Llama-3-8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,  # Allow using code that was not written by HuggingFace
    attn_implementation="flash_attention_2"  # Optimize the model with Flash Attention
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(DSDIR / "HuggingFace_Models/meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token_id = tokenizer.eos_token_id

def generation(prompt, **gen_parameters):
    """Generate text from a prompt and print it."""
    model_inp = tokenizer(prompt, return_tensors="pt").to("cuda")
    # the generate() method is a succession of forward (auto-regressive) 
    out = model.generate(input_ids=model_inp["input_ids"], pad_token_id=tokenizer.pad_token_id, **gen_parameters)
    return tokenizer.decode(out[0])

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def gen_quest_ans(utterence, speaker, emotion):
    prompt1 = f"""Write a question and an answer that ask who is the speaker or what is his emotion.
Context: The speaker is Monica. He or she is sadness. He or she said "I'm gonna miss you!".
Question:
Who is the speaker in this dialogue?
Answer:
Monica, who is saying "I'm gonna miss you!" and expressing sadness.<|end_of_text|>
Context: The speaker is {speaker}. He or she is {emotion}. He or she said "{utterence}".
Question:"""

    prompt2 = f"""Write a question and an answer that ask who is the speaker or what is his emotion.
Context: The speaker is Chandler. The emotion of the speach is joy. He or she said "You're a genius!".
Question:
What is the speaker's emotion?
Answer:
The speaker's emotion is Joy.<|end_of_text|>
Context: The speaker is {speaker}. He or she is {emotion}. He or she said "{utterence}".
Question:"""

    prompt = random.choice([prompt1, prompt2])
    retry = True

    while retry:
        try:
            out = generation(prompt, do_sample=True, max_new_tokens=300)
            pattern = r"""Question:.*Question:([A-Za-zà-ÿ\s0-9,.;:?!'"]*).*Answer:"""
            reg_match = re.search(pattern, out, re.DOTALL)
            question = reg_match[1].strip()
        
            pattern = r"""Answer:.*Answer:([A-Za-zà-ÿ\s0-9,.;:?!'"]*).*"""
            reg_match = re.search(pattern, out, re.DOTALL)
            answer = reg_match[1].strip()
            retry = False

        except:
            retry = True
            
    return question, answer

In [23]:
idx = 4
gen_quest_ans(df.loc[idx]["Utterance"], df.loc[idx]["Speaker"], df.loc[idx]["Emotion"])

('Who is the speaker in this dialogue?',
 'Joey, who is expressing sadness and saying "Aww, man, now we won\'t be bank buddies!"')

In [24]:
loop = tqdm(range(len(df)))

for idx in loop:
    question, answer = gen_quest_ans(df.loc[idx]["Utterance"], df.loc[idx]["Speaker"], df.loc[idx]["Emotion"])
    df.loc[idx, "question"] = question
    df.loc[idx, "answer"] = answer

df

  0%|          | 0/1109 [00:00<?, ?it/s]

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,question,answer
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049",What is the speaker's emotion?,The speaker's emotion is Sadness.
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261",Who is the speaker in this dialogue?,"Monica, who is saying ""What?"" and expressing s..."
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915",Who is the speaker?,The speaker is Ross.
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960",Who is the speaker in this dialogue?,"Chandler, who is expressing joy when he or she..."
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505",Who is the speaker?,The speaker is Joey.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,1174,No.,Monica,sadness,negative,113,9,6,2,"00:19:28,792","00:19:29,876","Who said ""No.""?","Monica, who said ""No."" and is expressing sadness."
1105,1175,What? Oh my God! Im gonna miss you so much!,Rachel,sadness,negative,113,10,6,2,"00:19:33,213","00:19:35,965",Who is the speaker?,The speaker is Rachel.
1106,1176,Im gonna miss you!,Monica,sadness,negative,113,11,6,2,"00:19:36,175","00:19:37,967",What is the speaker's emotion in this dialogue?,The speaker is expressing sadness.
1107,1177,I mean its the end of an era!,Rachel,sadness,negative,113,12,6,2,"00:19:39,094","00:19:40,928",Who is the speaker?,The speaker is Rachel.
